In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

print('Connected to MongoDB')


Connected to MongoDB


In [21]:
collection.delete_many({})
print("Koleksi 'employees' berhasil dikosongkan")

Koleksi 'employees' berhasil dikosongkan


In [22]:
# Menambahkan beberapa data karyawan
employees = [
    {'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500},
    {'name': 'Abimanyu', 'department': 'HR', 'age': 28, 'salary': 4200},
    {'name': 'Erlyn', 'department': 'IT', 'age': 31, 'salary': 5000},
    {'name': 'Sinta', 'department': 'Finance', 'age': 26, 'salary': 4000},
    {'name': 'Raka', 'department': 'IT', 'age': 27, 'salary': 4800}
]
collection.insert_many(employees)
print("Data awal berhasil ditambahkan")


Data awal berhasil ditambahkan


In [23]:
# Update semua karyawan di departemen IT jadi gaji 5200
collection.update_many(
    {'department': 'IT'},
    {'$set': {'salary': 5200}}
)
print("Data departemen IT berhasil di-update")

Data departemen IT berhasil di-update


In [24]:
# Hapus karyawan 
collection.delete_one({'name': 'Raka'})
print("Karyawan dengan nama raka berhasil dihapus")


Karyawan dengan nama raka berhasil dihapus


In [25]:
print("\nData Karyawan Saat Ini:")
data = list(collection.find())
print("Jumlah data:", len(data))
for employee in data:
    print(employee)


Data Karyawan Saat Ini:
Jumlah data: 4
{'_id': ObjectId('6903a089ae28424264e4ffa1'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('6903a089ae28424264e4ffa2'), 'name': 'Abimanyu', 'department': 'HR', 'age': 28, 'salary': 4200}
{'_id': ObjectId('6903a089ae28424264e4ffa3'), 'name': 'Erlyn', 'department': 'IT', 'age': 31, 'salary': 5200}
{'_id': ObjectId('6903a089ae28424264e4ffa4'), 'name': 'Sinta', 'department': 'Finance', 'age': 26, 'salary': 4000}


In [15]:
#query agregasi
pipeline = [
    {'$group': {'_id': '$department', 'average_salary': {'$avg': '$salary'}}}
]

print("\nRata-rata Gaji per Departemen:")
for result in collection.aggregate(pipeline):
    print(result)


Rata-rata Gaji per Departemen:
{'_id': 'Finance', 'average_salary': 4250.0}
{'_id': 'HR', 'average_salary': 4200.0}
{'_id': 'IT', 'average_salary': 5200.0}


In [6]:
#nomor 4.1
# Buat koleksi baru untuk produk
products = db['products']

# Hapus semua data lama agar bersih
products.delete_many({})

# Tambahkan data produk bahan pokok
product_data = [
    {'name': 'Beras Premium 5kg', 'category': 'Bahan Pokok', 'price': 75000, 'stock': 20},
    {'name': 'Gula Pasir 1kg', 'category': 'Bahan Pokok', 'price': 16000, 'stock': 50},
    {'name': 'Minyak Goreng 1L', 'category': 'Bahan Pokok', 'price': 21000, 'stock': 30},
    {'name': 'Telur Ayam 1kg', 'category': 'Bahan Pokok', 'price': 28000, 'stock': 40},
    {'name': 'Sabun Mandi', 'category': 'Kebutuhan Rumah Tangga', 'price': 12000, 'stock': 25},
    {'name': 'Sampo 200ml', 'category': 'Kebutuhan Rumah Tangga', 'price': 18000, 'stock': 15},
    {'name': 'Tepung Terigu 1kg', 'category': 'Bahan Pokok', 'price': 14000, 'stock': 35},
    {'name': 'Air Mineral 1L', 'category': 'Minuman', 'price': 8000, 'stock': 100},
    {'name': 'Kopi Bubuk 200g', 'category': 'Minuman', 'price': 25000, 'stock': 20},
    {'name': 'Teh Celup 25pcs', 'category': 'Minuman', 'price': 18000, 'stock': 40}
]

products.insert_many(product_data)
print("Data produk berhasil ditambahkan.")



Data produk berhasil ditambahkan.


In [5]:
# Hitung rata-rata harga semua produk
avg_price = list(products.aggregate([
    {'$group': {'_id': None, 'avgPrice': {'$avg': '$price'}}}
]))[0]['avgPrice']

print(f"\nRata-rata harga semua produk: {avg_price:.2f}")

# Cari produk dengan harga di atas rata-rata
print("\nProduk dengan harga di atas rata-rata:")
for p in products.find({'price': {'$gt': avg_price}}):
    print(p)



Rata-rata harga semua produk: 23500.00

Produk dengan harga di atas rata-rata:
{'_id': ObjectId('690af791f172d670405651e0'), 'name': 'Beras Premium 5kg', 'category': 'Bahan Pokok', 'price': 75000, 'stock': 20}
{'_id': ObjectId('690af791f172d670405651e3'), 'name': 'Telur Ayam 1kg', 'category': 'Bahan Pokok', 'price': 28000, 'stock': 40}
{'_id': ObjectId('690af791f172d670405651e8'), 'name': 'Kopi Bubuk 200g', 'category': 'Minuman', 'price': 25000, 'stock': 20}


In [7]:
# Aggregation: Total produk dalam setiap kategori
pipeline = [
    {'$group': {'_id': '$category', 'total_products': {'$sum': 1}}},
    {'$sort': {'total_products': -1}}
]

print("\nTotal Produk per Kategori:")
for result in products.aggregate(pipeline):
    print(result)



Total Produk per Kategori:
{'_id': 'Bahan Pokok', 'total_products': 5}
{'_id': 'Minuman', 'total_products': 3}
{'_id': 'Kebutuhan Rumah Tangga', 'total_products': 2}


In [9]:
print('1. Cari 5 karyawan dengan gaji tertinggi dalam setiap departemen')

pipeline = [
    {'$sort': {'department': 1, 'salary': -1}},   # Urutkan per departemen dan gaji tertinggi
    {'$group': {
        '_id': '$department',
        'top_employees': {
            '$push': {'name': '$name', 'salary': '$salary'}
        }
    }},
    {'$project': {
        'top_5': {'$slice': ['$top_employees', 5]}  # Ambil 5 teratas per departemen
    }}
]

print("5 Karyawan dengan Gaji Tertinggi per Departemen:\n")
for result in collection.aggregate(pipeline):
    print(result)


1. Cari 5 karyawan dengan gaji tertinggi dalam setiap departemen
5 Karyawan dengan Gaji Tertinggi per Departemen:

{'_id': 'IT', 'top_5': [{'name': 'Erlyn', 'salary': 5200}]}
{'_id': 'Finance', 'top_5': [{'name': 'Alice', 'salary': 4500}, {'name': 'Sinta', 'salary': 4000}]}
{'_id': 'HR', 'top_5': [{'name': 'Abimanyu', 'salary': 4200}]}


In [12]:
print('2. Menghapus karyawan dengan usia di bawah 25 tahun')
delete_result = collection.delete_many({'age': {'$lt': 25}})
print(f"\nJumlah karyawan di bawah 25 tahun yang dihapus: {delete_result.deleted_count}")

# Cek data setelah penghapusan
print("\nData Karyawan Setelah Penghapusan:")
for emp in collection.find():
    print(emp)

2. Menghapus karyawan dengan usia di bawah 25 tahun

Jumlah karyawan di bawah 25 tahun yang dihapus: 0

Data Karyawan Setelah Penghapusan:
{'_id': ObjectId('6903a089ae28424264e4ffa1'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('6903a089ae28424264e4ffa2'), 'name': 'Abimanyu', 'department': 'HR', 'age': 28, 'salary': 4200}
{'_id': ObjectId('6903a089ae28424264e4ffa3'), 'name': 'Erlyn', 'department': 'IT', 'age': 31, 'salary': 5200}
{'_id': ObjectId('6903a089ae28424264e4ffa4'), 'name': 'Sinta', 'department': 'Finance', 'age': 26, 'salary': 4000}


In [13]:
print('3. Laporan ringkas total gaji dan rata-rata umur per departemen')

pipeline = [
    {'$group': {
        '_id': '$department',
        'total_salary': {'$sum': '$salary'},
        'average_age': {'$avg': '$age'},
        'total_employees': {'$sum': 1}
    }},
    {'$sort': {'_id': 1}}
]

print("\nLaporan Ringkas per Departemen:")
for result in collection.aggregate(pipeline):
    print(result)


3. Laporan ringkas total gaji dan rata-rata umur per departemen

Laporan Ringkas per Departemen:
{'_id': 'Finance', 'total_salary': 8500, 'average_age': 27.5, 'total_employees': 2}
{'_id': 'HR', 'total_salary': 4200, 'average_age': 28.0, 'total_employees': 1}
{'_id': 'IT', 'total_salary': 5200, 'average_age': 31.0, 'total_employees': 1}
